In [ ]:
spark.sparkContext._jsc.hadoopConfiguration().get("fs.s3a.endpoint")


In [1]:
# Helper para criação de tabelas
from pyiceberg.partitioning import PartitionSpec
from pyiceberg.transforms import IdentityTransform
from pyiceberg.schema import Schema
from pyiceberg.types import NestedField
from pyiceberg.io.pyarrow import pyarrow_to_schema


def build_partition_spec(schema, partition_cols):
    # nenhum particionamento
    if not partition_cols:
        return PartitionSpec()

    fields = []

    for i, col in enumerate(partition_cols, start=1000):
        field = schema.find_field(col)

        if field is None:
            raise ValueError(f"Coluna '{col}' não existe no schema")

        fields.append(
            PartitionSpec.Field(
                source_id=field.field_id,
                field_id=i,
                transform=IdentityTransform(),
                name=col,
            )
        )

    return PartitionSpec(*fields)


def update_schema(table, arrow_schema):
    """
    Atualiza o schema da tabela Iceberg se houver colunas novas no DataFrame.
    
    Parameters
    ----------
    table : Iceberg table
        Tabela a ser verificada/atualizada
    arrow_schema : pyarrow.Schema
        Schema do DataFrame que será inserido
        
    Returns
    -------
    bool
        True se o schema foi atualizado, False caso contrário
    """ 
    # Atualiza o schema adicionando as novas colunas
    with table.update_schema() as update:
        update.union_by_name(arrow_schema)
    
    print(f"✔ schema atualizado")
    return True

def set_table(
    catalog,
    table_name: str,
    df,
    partition_cols=None,
    mode="append",
):
    """
    Cria ou carrega uma tabela Iceberg e escreve dados nela.

    Parameters
    ----------
    catalog : Iceberg catalog
    table_name : str
    df : polars.DataFrame
    partition_cols : list[str] | None
    mode : 'append' | 'overwrite'
    """

    # garante nomes limpos
    df = df.rename(lambda c: c.strip())

    arrow_table = df.to_arrow()
    arrow_schema = arrow_table.schema

    # -------------------
    # load or create
    # -------------------
    try:
        table = catalog.load_table(table_name)
        update_schema(table, arrow_schema)
        print(f"✔ tabela '{table_name}' carregada")

    except Exception as e:
        if "NoSuchTable" not in str(type(e)):
            raise  # erro real → não cria tabela
    
        print(f"⚙ criando tabela '{table_name}'")
    
        partition_spec = build_partition_spec(arrow_schema, partition_cols)
    
        table = catalog.create_table(
            identifier=table_name,
            schema=arrow_schema,
            partition_spec=partition_spec,
            properties={
                "write.format.default": "parquet",
                "write.parquet.compression-codec": "zstd",
                "schema.name-mapping.default": "true",
            },
        )
    # -------------------
    # write
    # -------------------
    if mode == "overwrite":
        table.overwrite(arrow_table)
    elif mode == "append":
        table.append(arrow_table)
    elif mode == "reset":
        catalog.drop_table(table_name)
        table = catalog.create_table(
            identifier=table_name,
            schema=arrow_schema,
        )
        table.append(arrow_table)

    print(f"✔ dados gravados ({mode})")

    return table

In [ ]:

hadoop_conf

In [ ]:
df_csv = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("s3a://raw/nubank/Nubank_2025-06-10.csv")

df_csv.show()

In [ ]:
# Use Python, não %%sql
spark.sql("""
    CREATE OR REPLACE TABLE dev.raw.nubank_faturas
    USING csv
    OPTIONS (
        path 's3://raw/nubank/',
        header 'true',
        inferSchema 'true'
    )
""")

In [ ]:
from pyspark.sql.functions import *

# 1️⃣ Ler CSV (via tabela externa Iceberg)
spark.sql("""
    CREATE OR REPLACE TABLE dev.raw.nubank_faturas
    USING csv
    OPTIONS (
        path 's3://raw/nubank/',
        header 'true',
        inferSchema 'true'
    )
""")

df_raw = spark.table("dev.raw.nubank_faturas")
print(f"📄 Total de registros RAW: {df_raw.count()}")

# 2️⃣ Transformar
df_silver = df_raw \
    .filter(col("_c0").isNotNull()) \
    .withColumn("processed_at", current_timestamp())

# 3️⃣ Salvar no Silver (agora como tabela Iceberg nativa)
df_silver.write \
    .format("iceberg") \
    .mode("overwrite") \
    .saveAsTable("demo.silver.nubank_faturas")

print("✅ Pipeline concluído!")

# 4️⃣ Consultar no Silver
spark.sql("SELECT * FROM dev.silver.nubank_faturas LIMIT 10").show()

In [ ]:
# Lê todos os arquivos CSV dentro da pasta nubank
path = "s3a://raw/nubank/*.csv"

df_raw = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("sep", ",") \
    .load(path)

In [ ]:
# Adiciona uma coluna para saber de qual arquivo veio o dado (muito útil para auditoria)
from pyspark.sql.functions import input_file_name
df_raw = df_raw.withColumn("source_file", input_file_name())

In [ ]:
df_raw.select("source_file").distinct()

In [ ]:
# Criando a base de dados no catálogo se não existir
spark.sql("CREATE DATABASE IF NOT EXISTS demo.bronze")

# Salvando como tabela Iceberg
df_raw.writeTo("dev.bronze.nubank_raw").createOrReplace()

In [ ]:
%%sql
select * from demo.bronze.nubank_raw order by amount desc

In [ ]:
from pyspark.sql.functions import col, to_date, regexp_replace

df_silver = spark.table("dev.bronze.nubank_raw") \
    .withColumn("data_transacao", to_date(col("date"), "yyyy-MM-dd")) \
    .withColumn("valor", col("amount").cast("double")) \
    .select(
        col("data_transacao"),
        col("valor"),
        col("title").alias("descricao"),
        col("source_file")
    )
df_silver.show(5)

In [ ]:


# Salva na camada Silver
spark.sql("CREATE DATABASE IF NOT EXISTS demo.silver")
df_silver.writeTo("dev.silver.nubank_clean").createOrReplace()

In [ ]:
from pyspark.sql.functions import col, to_date, regexp_replace

df_silver = spark.table("dev.bronze.nubank_raw") \
    .withColumn("data_transacao", to_date(col("Data"), "dd/MM/yyyy")) \
    .withColumn("valor", regexp_replace(col("Valor"), ",", ".").cast("double")) \
    .select(
        col("data_transacao"),
        col("valor"),
        col("Identificador").alias("id_transacao"),
        col("Descrição").alias("descricao")
    )

# Salva na camada Silver
spark.sql("CREATE DATABASE IF NOT EXISTS demo.silver")
df_silver.writeTo("dev.silver.nubank_clean").createOrReplace()

In [ ]:
categorias = [
    ("iFood", "Alimentação"),
    ("Uber", "Transporte"),
    ("Posto", "Transporte"),
    ("Tozetto", "Alimentação"),
    ("Muffato", "Alimentação"),
    ("Condor", "Alimentação"),
    ("Pizza", "Alimentação"),
    ("Farma", "Saúde"),
]
# Transformando em DataFrame
df_mapping = spark.createDataFrame(categorias, ["estabelecimento_keyword", "categoria_final"])

# Salvando no Iceberg para uso futuro (Camada Silver ou Ref)
df_mapping.writeTo("dev.silver.mapping_nubank_categorias").createOrReplace()


In [ ]:
%%sql
    select 
    sum(valor) as valor
    , categoria_final as categoria 
    , concat(month(data_transacao), "/", year(data_transacao)) as mes_ano 
    from dev.silver.nubank_clean 
    left join dev.silver.mapping_nubank_categorias
        on (lower(descricao) like concat('%', trim(lower(estabelecimento_keyword)),'%'))
    group by mes_ano, categoria

In [ ]:
import duckdb

con = duckdb.connect()

# carrega a extensão Iceberg
con.execute("""
    INSTALL httpfs;
    INSTALL iceberg;
    UPDATE EXTENSIONS;
    LOAD iceberg;
    LOAD httpfs;
  ATTACH 'warehouse' AS my_r2_catalog (
    TYPE iceberg,
    SECRET 'DUMMY',
    ENDPOINT http://rest:8181'
);
""")

df_joined = con.execute("""
    SELECT e.*
    FROM read_iceberg('silver.nubank_clean') AS e
    LIMIT 5
""").fetch_df()

In [ ]:
con.execute("""
    SHOW ALL TABLES;
""")

In [ ]:
# Carrega sua tabela silver que acabamos de processar
from pyspark.sql.functions import lower, col

df_mapping = spark.table("dev.silver.mapping_nubank_categorias")

df_transacoes = spark.table("dev.silver.nubank_clean")
df_categorias = df_transacoes.join(
    df_mapping,
    lower(col("descricao")).contains(lower(col("estabelecimento_keyword"))),
    "left"
).select(
    df_transacoes["*"],
    col("categoria_final").alias("categoria")
).filter(col("data_transacao") > "2025-01-01")

df_categorias = df_categorias.fillna({"categoria": "Uncategorized"})

nulas = df_categorias.where(df_categorias.categoria == "Uncategorized")

print(f"{nulas.count()} itens restantes")
nulas.orderBy("descricao", ascending=True).limit(20).toPandas()

In [ ]:
from pyspark.sql.functions import sum

df_categorias.groupBy("categoria").agg(
    sum("valor").alias("total")   
).orderBy("total", ascending=False).toPandas()

In [ ]:
%%sql
INSERT INTO demo.silver.mapping_nubank_categorias VALUES
      -- ('Pg Triciclos e Bicicl', 'Alimentação')
      -- ,('Luis Henrique de Jesus Rodrigues', 'Casa')
      -- ,('Blinkapp', 'Assinaturas')
      -- ,('prihair', 'Acessórios')
      -- ,('potencial gas', 'Casa')
-- UPDATE dev.silver.mapping_nubank_categorias SET categoria_final='Celular' WHERE categoria_final='Celulares'

In [ ]:
%%sql
  select * from demo.silver.mapping_nubank_categorias 
    where Lower(estabelecimento_keyword) LIKE '%google%'

In [ ]:
%%sql
select distinct(categoria_final)  from dev.silver.mapping_nubank_categorias 

In [ ]:
spark.table("dev.silver.mapping_nubank_categorias").count()

In [ ]:
!pip install polars


In [ ]:
%%sql
    CALL dev.system.register_table(
  table => 'silver.mapping_nubank_categorias',
  metadata_file => 's3://warehouse/silver/mapping_nubank_categorias/metadata/00063-b274efd4-b285-4738-b862-35cfd31c2bc4.metadata.json'
)
--     ;
--     SELECT * 
-- FROM iceberg.`s3a://warehouse/bronze/nubank_raw/metadata/00000-418fe01e-b014-42b9-b94d-790ca29408e2.metadata.json`


In [ ]:
%%sql
CALL dev.system.rewrite_manifests('silver.mapping_nubank_categorias')


In [ ]:
%%sql
show catalogs

In [ ]:
from pyiceberg.catalog import load_catalog
import polars as pl


In [ ]:

catalog = load_catalog(
    "dev",
    uri="http://rest:8181",
    s3_endpoint="http://minio:9000",
    s3_access_key_id="minioadmin",
    s3_secret_access_key="minioadmin",
)
table = catalog.load_table("silver.mapping_nubank_categorias")
# table = catalog.load_table("default.taxis")
arrow_table = table.scan().to_arrow()
df = pl.from_arrow(arrow_table)
df


In [ ]:
import boto3

sts = boto3.client("sts")
sts.get_caller_identity()

In [5]:
from pyiceberg.catalog import load_catalog
import polars as pl

# -----------------------------
# 1️⃣ Carrega catálogo Iceberg
# -----------------------------
# catalog = load_catalog("default")  # seu catálogo já configurado (REST/Hive/etc)
catalog = load_catalog(
    "dev",
    uri="http://rest:8181",

    **{
        "s3.endpoint": "http://minio:9000",
        "s3.access-key-id": "minioadmin",
        "s3.secret-access-key": "minioadmin",
        "s3.path-style-access": "true",
    }
)


# -----------------------------
# 2️⃣ Carrega tabelas Iceberg
# -----------------------------
table_clean = catalog.load_table("silver.nubank_clean")
table_mapping = catalog.load_table("silver.mapping_nubank_categorias")

# -----------------------------
# 3️⃣ Cria LazyFrames Polars
# -----------------------------
df_clean = pl.scan_iceberg(table_clean, storage_options=catalog.properties)
df_mapping = pl.scan_iceberg(table_mapping, storage_options=catalog.properties)


In [6]:
# -----------------------------
# 4️⃣ Cria coluna mes_ano
# -----------------------------
df_clean = df_clean.with_columns(
    mes_ano = (
        pl.col("data_transacao").dt.year().cast(pl.Utf8) + "/" +
        pl.col("data_transacao").dt.month().cast(pl.Utf8) + '/01'
    ).str.to_date()
)

# -----------------------------
# 5️⃣ Join via LIKE (lazy)
# -----------------------------
# Observação: Polars não tem join LIKE nativo, usamos cross join + filter
df_joined = (
    df_clean.join(df_mapping, how="cross")
    .filter(
        pl.col("descricao")
        .str.to_lowercase()
        .str.contains(
            pl.col("estabelecimento_keyword")
            .str.strip_chars()
            .str.to_lowercase()
        )
    )
)

# -----------------------------
# 6️⃣ Agregação
# -----------------------------
df_result = df_joined.group_by(["mes_ano", "categoria_final"]).agg(
    pl.sum("valor").alias("valor")
)

# -----------------------------
# 7️⃣ Preview seguro
# -----------------------------
print(df_result.limit(10).collect())

catalog.drop_table('silver.nubank_agg')
set_table(catalog, 
          "silver.nubank_agg",
          df_result.collect(),
          mode="reset"
         )

# -----------------------------
# 8️⃣ Preparar para salvar no Iceberg
# -----------------------------
# Converte para Arrow Table (materializa aqui apenas)

# arrow_table = df_result.collect().to_arrow()

# # Carrega tabela destino e escreve (append ou nova tabela)
# table_result = catalog.load_table("silver.nubank_agg")
# from pyiceberg.io.pyarrow import write_table
# write_table(arrow_table, table_result)

shape: (10, 3)
┌────────────┬─────────────────────────────┬──────────┐
│ mes_ano    ┆ categoria_final             ┆ valor    │
│ ---        ┆ ---                         ┆ ---      │
│ date       ┆ str                         ┆ f64      │
╞════════════╪═════════════════════════════╪══════════╡
│ 2025-05-01 ┆ Transporte                  ┆ 595.76   │
│ 2025-08-01 ┆ Impostos                    ┆ 77.4     │
│ 2025-08-01 ┆ Outras transações de fatura ┆ -4531.54 │
│ 2025-05-01 ┆ Pet                         ┆ 93.19    │
│ 2025-05-01 ┆ Acessórios                  ┆ 240.7    │
│ 2025-02-01 ┆ Viagens                     ┆ 247.63   │
│ 2025-11-01 ┆ Alimentação                 ┆ 1504.34  │
│ 2025-10-01 ┆ Assinaturas                 ┆ 636.78   │
│ 2025-07-01 ┆ Transporte                  ┆ 500.48   │
│ 2025-11-01 ┆ Pet                         ┆ 32.99    │
└────────────┴─────────────────────────────┴──────────┘
⚙ criando tabela 'silver.nubank_agg'
✔ dados gravados (reset)


nubank_agg(
  1: mes_ano: optional date,
  2: categoria_final: optional string,
  3: valor: optional double
),
partition by: [],
sort order: [],
snapshot: Operation.APPEND: id=5270333597873194744, schema_id=0

In [99]:
tbl = catalog.load_table("silver.nubank_agg")

# df = pl.from_arrow(tbl.scan().to_arrow())
df = pl.scan_iceberg(tbl, storage_options=catalog.properties)


print(df.collect())

shape: (136, 3)
┌─────────┬─────────────────────────────┬─────────┐
│ mes_ano ┆ categoria_final             ┆ valor   │
│ ---     ┆ ---                         ┆ ---     │
│ str     ┆ str                         ┆ f64     │
╞═════════╪═════════════════════════════╪═════════╡
│ 9/2025  ┆ Casa                        ┆ 467.07  │
│ 2/2025  ┆ Alimentação                 ┆ 3118.22 │
│ 1/2025  ┆ Transporte                  ┆ 323.57  │
│ 1/2025  ┆ Outras transações de fatura ┆ -4409.9 │
│ 3/2025  ┆ Impostos                    ┆ 254.13  │
│ …       ┆ …                           ┆ …       │
│ 7/2025  ┆ Pesquisa Acadêmica          ┆ 427.0   │
│ 6/2025  ┆ Celular                     ┆ 295.76  │
│ 4/2025  ┆ Casa                        ┆ 171.24  │
│ 4/2025  ┆ Saúde                       ┆ 358.45  │
│ 10/2025 ┆ Assinaturas                 ┆ 636.78  │
└─────────┴─────────────────────────────┴─────────┘
